In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qjGe6f94aB2ZG1cXX0zVLumR73GVVXox5h3H-_Y-cNtv6vuxHOaqE0



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [57]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# load data

In [5]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

In [137]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 
print(boundary_georef.to_string())

             geo_name   level aoi_boundary_name units_boundary_name
0         ARG-Mendoza  region         ADM3union                ADM3
1   ARG-Mar_del_Plata    city              ADM3                ADM4
2   ARG-Mar_del_Plata  region              ADM2                 NaN
3         ARG-Ushuaia    city              ADM4                ADM5
4         ARG-Ushuaia  region              ADM3                 NaN
5           ARG-Salta  region         ADM2union                ADM3
6    ARG-Buenos_Aires  region         ADM2union                ADM2
7        BRA-Teresina    city         ADM4union                ADM4
8        BRA-Teresina  region         ADM2union                ADM2
9   BRA-Florianopolis    city         ADM4union                ADM4
10  BRA-Florianopolis  region         ADM2union                ADM2
11          BRA-Belem    city         ADM4union                ADM4
12          BRA-Belem  region         ADM2union                ADM2
13       CRI-San_Jose  region         ADM2union 

# Extract ESA raster by feature

In [32]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for Land cover as GeoTIFF
    geemap.ee_export_image_to_drive(
        esa_land_cover, 
        description = boundary_id + '-ESA-Landcover',
        folder='data', 
        scale=10, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

0
ARG-Mendoza-ADM3union
1
ARG-Mar_del_Plata-ADM3
2
ARG-Mar_del_Plata-ADM2
3
ARG-Ushuaia-ADM4
4
ARG-Ushuaia-ADM3
5
ARG-Salta-ADM2union
6
ARG-Buenos_Aires-ADM2union
7
BRA-Teresina-ADM4union
8
BRA-Teresina-ADM2union
9
BRA-Florianopolis-ADM4union
10
BRA-Florianopolis-ADM2union
11
BRA-Belem-ADM4union
12
BRA-Belem-ADM2union
13
CRI-San_Jose-ADM2union
14
RWA-Kigali-ADM4union
15
SLE-Freetown-ADM4cityunion
16
SLE-Freetown-ADM4regionunion
17
MAR-Marrakech-ADM2
18
IND-Chennai-ADM4union
19
IND-Chennai-ADM6union
20
IND-Pune-ADM4union
21
IND-Surat-ADM4union
22
CHN-Chengdu-ADM3union
23
CHN-Chongqing-ADM1
24
CHN-Ningbo-ADM3union
25
IDN-Jakarta-ADM4union
26
IDN-Bitung-ADM2
27
IDN-Semarang-ADM1
28
IDN-Balikpapan-ADM4union
29
IDN-Palembang-ADM2


# upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [39]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-urbanshift' 

In [36]:
out_dir = os.getcwd()

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)

    # read local raster
    city_file = 'data\\' + geo_name +'_ESA_Worldcover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/land_use/esa_world_cover/v_0/'+ boundary_id + '-ESA-world_cover-2000.tif',
                               ExtraArgs={'ACL':'public-read'})